<a href="https://colab.research.google.com/github/CEOApplepine/mega-data-engineering-project/blob/main/02_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: Install packages
!pip install pandas pyspark --quiet

# Step 2: Import libraries
import pandas as pd
from pyspark.sql import SparkSession

# Step 3: Initialize Spark
spark = SparkSession.builder.appName("MegaDataProject").getOrCreate()

# Step 4: Download datasets locally
!wget -O taxi.csv https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv
!wget -O retail.csv https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv

# Step 5: Load CSV into Spark DataFrames (local files)
taxi_sdf = spark.read.option("header", True).csv("taxi.csv")
retail_sdf = spark.read.option("header", True).csv("retail.csv")

# Step 6: Show sample data
taxi_sdf.show(5)
retail_sdf.show(5)

--2025-11-22 20:47:15--  https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 321 [text/csv]
Saving to: ‘taxi.csv’

taxi.csv            100%[===================>]     321  --.-KB/s    in 0s      

2025-11-22 20:47:15 (147 MB/s) - ‘taxi.csv’ saved [321/321]

--2025-11-22 20:47:15--  https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328 [text/csv]
Saving to: ‘retail.csv’

retail.csv          100%[===================>]     328  --.-KB/s    in 0s      

2025-11-22 20:47:15 (218 MB/s) - ‘retail.csv’ saved [328/328]

+-----+-------+-------+-------+
|Month| "1958"| "1959"| "196

In [7]:
# ===========================
# 02_BigData.ipynb - Colab-ready
# ===========================

# Step 1: Install required packages
!pip install pandas pyspark duckdb --quiet

# Step 2: Import libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum as spark_sum

# Step 3: Initialize Spark
spark = SparkSession.builder.appName("MegaDataProject").getOrCreate()

# Step 4: Download datasets locally
!wget -O taxi.csv https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv
!wget -O retail.csv https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv

# Step 5: Load CSV into Spark DataFrames
taxi_sdf = spark.read.option("header", True).csv("taxi.csv")
retail_sdf = spark.read.option("header", True).csv("retail.csv")

# Step 6: Show sample data
print("Taxi Dataset Sample:")
taxi_sdf.show(5)

print("Retail Dataset Sample:")
retail_sdf.show(5)

# Step 7: Basic Data Cleaning / Transformations
# Convert month columns to integers in taxi dataset
taxi_sdf = taxi_sdf.withColumnRenamed("Month", "Month_Name")
for col_name in taxi_sdf.columns[1:]:
    taxi_sdf = taxi_sdf.withColumn(col_name, col(col_name).cast("int"))

print("Taxi Dataset Schema:")
taxi_sdf.printSchema()

# Step 8: Example Join (for demo purposes)
taxi_sdf = taxi_sdf.withColumn("key", lit(1))
retail_sdf = retail_sdf.withColumn("key", lit(1))

joined_df = taxi_sdf.join(retail_sdf, on="key").drop("key")
print("Joined Dataset Sample:")
joined_df.show(5)

# Step 9: Aggregations / Analysis
month_cols = taxi_sdf.columns[1:]  # skip Month_Name
taxi_sdf.createOrReplaceTempView("taxi")
total_passengers = spark.sql(
    "SELECT Month_Name, " + " + ".join(month_cols) + " AS Total_Passengers FROM taxi"
)
print("Total Passengers per Month:")
total_passengers.show()

# Step 10: Save cleaned datasets to CSV in Colab (optional)
taxi_sdf.toPandas().to_csv("taxi_cleaned.csv", index=False)
retail_sdf.toPandas().to_csv("retail_cleaned.csv", index=False)
print("Cleaned datasets saved locally in Colab.")

--2025-11-22 20:50:11--  https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 321 [text/csv]
Saving to: ‘taxi.csv’

taxi.csv            100%[===================>]     321  --.-KB/s    in 0s      

2025-11-22 20:50:11 (154 MB/s) - ‘taxi.csv’ saved [321/321]

--2025-11-22 20:50:11--  https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv
Resolving people.sc.fsu.edu (people.sc.fsu.edu)... 144.174.0.22
Connecting to people.sc.fsu.edu (people.sc.fsu.edu)|144.174.0.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328 [text/csv]
Saving to: ‘retail.csv’

retail.csv          100%[===================>]     328  --.-KB/s    in 0s      

2025-11-22 20:50:11 (216 MB/s) - ‘retail.csv’ saved [328/328]

Taxi Dataset Sample:
+-----+-------+-------+-------+
|Month|